In [65]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import unidecode
import os
import re
import warnings

warnings.filterwarnings("ignore")

# Cargar datasets preprocesados
df_completo = pd.read_parquet(r'../Datasets/dataset_completo.parquet')

def limpiar_texto_completo(texto):
    if pd.isnull(texto):
        return None
    try:
        texto = str(texto).lower().strip()  # Convertir a minúsculas y eliminar espacios al inicio y al final
        texto = ' '.join(texto.split())  # Normalizar espacios adicionales
        texto = unidecode.unidecode(texto)  # Eliminar acentos y caracteres especiales
        # Eliminar solo números aislados al principio del título, pero no secuencias como "1991" dentro del cuerpo del título
        texto = re.sub(r'^\d+\s+', '', texto)  # Eliminar números si están seguidos por un espacio
        texto = re.sub(r'[^A-Za-z0-9\s.,\'-]', '', texto)  # Eliminar símbolos especiales
        texto = texto.strip()  # Eliminar espacios residuales tras la limpieza
    except Exception as e:
        print(f"Error procesando el texto: {texto}, Error: {e}")
        return None 
    return texto


df_completo['title_normalizado'] = df_completo['title'].apply(limpiar_texto_completo) # actualizar

# Verificar si 'title_normalizado' se ha creado correctamente
if 'title_normalizado' not in df_completo.columns:
    raise ValueError("La columna 'title_normalizado' no se ha creado correctamente.")

# Crear la matriz de características basada en las columnas relevantes
def generar_features_matrix(df):
    # Selecciona las columnas relevantes para la recomendación
    features = df[['vote_average', 'vote_count', 'popularity']].fillna(0)  # Asegurar que no haya valores nulos
    
    # Normaliza las columnas seleccionadas
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    
    return features_scaled

# Generar la matriz de características en memoria
features_matrix = generar_features_matrix(df_completo)

# Verificar si la longitud de 'features_matrix' coincide con 'df_completo'
if len(features_matrix) != len(df_completo):
    raise ValueError("La longitud de la matriz de características no coincide con la longitud del DataFrame.")

# Guardar la matriz de características si es necesario
features_df = pd.DataFrame(features_matrix, columns=['vote_average', 'vote_count', 'popularity'])
features_df.to_parquet('../Datasets/features_matrix.parquet', index=False)

# Función de recomendación basada en la similitud del coseno
def recomendacion(titulo, n_recomendaciones=5):
    try:
        # Limpiar y normalizar el título de la película
        titulo_normalizado = limpiar_texto_completo(titulo)
        
        # Verificar si el título existe en el dataset
        pelicula_idx = df_completo[df_completo['title_normalizado'].str.contains(titulo_normalizado, case=False, na=False)].index
        
        if pelicula_idx.empty:
            return {"mensaje": f"El título '{titulo}' no se encuentra en el dataset."}
        
        # Obtener el índice de la película en el dataset
        idx = pelicula_idx[0]
        
        # Calcular las similitudes utilizando la matriz de características
        cosine_similarities = cosine_similarity([features_matrix[idx]], features_matrix).flatten()
        
        # Verificar que el número de recomendaciones sea válido
        n_recomendaciones = min(n_recomendaciones, len(cosine_similarities) - 1)
        
        # Obtener los índices de las películas más similares y evitar repetir la misma película
        similar_indices = cosine_similarities.argsort()[::-1][1:n_recomendaciones + 1]
        
        # Obtener los títulos de las películas recomendadas
        recomendaciones = df_completo['title'].iloc[similar_indices].tolist()
        
        return {"recomendaciones": recomendaciones}
    
    except Exception as e:
        return {"mensaje": f"Error interno en la recomendación: {str(e)}"}

# Probar la función de recomendación
resultado = recomendacion("a passage to india")
print(resultado)


{'recomendaciones': ['motherhood', 'bratz', 'fandango', 'i am omega', 'kung fu dunk']}


In [66]:
# Verifica algunos valores para asegurarte de que están correctos
print(df_completo[['title', 'title_normalizado']].head(10))


                             title             title_normalizado
0  10 questions for the dalai lama  questions for the dalai lama
1                         11-11-11                      11-11-11
2                           131313                        131313
3              16 years of alcohol              years of alcohol
4              18 fingers of death              fingers of death
5         1991 the year punk broke           the year punk broke
6                    24 redemption                    redemption
7                   28 hotel rooms                   hotel rooms
8                     29th and gay                  29th and gay
9              3 ninjas knuckle up             ninjas knuckle up
